In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import itertools
import os
import time
from scipy import stats
from scipy import constants
from scipy.integrate import odeint
from scipy.optimize import curve_fit



from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV, train_test_split, GridSearchCV
from sklearn.preprocessing import  MinMaxScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor, StackingRegressor, VotingRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import mean_squared_error

import torch
import torch.nn as nn
from torch.nn import ReLU
import torch.utils.data as td

# Set random seed for reproducability
torch.manual_seed(0)

print("Libraries imported - ready to use PyTorch", torch.__version__)

Libraries imported - ready to use PyTorch 1.12.1+cpu


In [ ]:
class Inverse(nn.Module):
        def __init__(self):
                super(Inverse, self).__init__()        
                self.LAMBDA = nn.Sequential(
                        nn.Linear(2, 32),
                        nn.ReLU(),
                        nn.Linear(32, 64),
                        nn.ReLU(),
                        nn.Linear(64, 1))

                self.stress = nn.Sequential(
                        nn.Linear(2, 32),
                        nn.ReLU(),
                        nn.Linear(32, 64),
                        nn.ReLU(),
                        nn.Linear(64, 1))
                        
                
        def forward(self,h,T):
                in_1=torch.concat((h,T),1)
                lam = self.LAMBDA(in_1) 
                in_2=torch.concat((lam,h),1) 
                T   = self.stress(in_2)  

                return lam,T          
model=Inverse()


In [ ]:
loss_criteria=nn.MSELoss()
## Trainning function definition
alpha=1
beta=1
def train(model,h,T,lam,optimizer):
  for i in range(len(h)):
    model.train()
    
    # for i in range(len(h)):
    # optimizer.zero_grad()
    out1,out2=model(h,T)
    loss1=loss_criteria(out1,lam)
    loss2=loss_criteria(out2,T)
    loss=alpha*loss1+beta*loss2
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  return loss,out1,out2

learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
##Saving the model state of 50 epoch

# Additional information
EPOCH = 50
PATH = "model_str.pt"
LOSS = 0.015409946

torch.save({
            'epoch': EPOCH,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': LOSS,
            }, PATH)

In [ ]:
##Loading model with 100 epoch

model = Inverse()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

model.eval()